# Version 2:

## About:
Notebook is used to generate the inputs for the ML model requirements. <br>
The output files for this are:
<ul>
    <li>team_stats.csv - aggregated stats to team level by round</li>
    <li>team_score_results.csv - stacked scored to team level by round</li>
    <li>merged_stat_score_data_clean.csv - combined dataset of aggregated team stats and team scores by round</li>
</ul>

In [65]:
import pandas as pd

In [66]:
from featuresv2 import features, targets, metadata

In [67]:
# read raw data
game_results = pd.read_csv("data/game_results.csv")
player_stats = pd.read_csv("data/player_stats.csv")

# add season field
player_stats['season'] = [i.strftime('%Y') for i in pd.to_datetime(player_stats['utcStartTime']).to_list()]

C:\Users\whitehes\AppData\Local\Temp\ipykernel_21384\3885183847.py:3: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  player_stats = pd.read_csv("data/player_stats.csv")


In [68]:
player_stats.head()

,providerId,utcStartTime,status,compSeason.shortName,round.name,round.roundNumber,venue.name,home.team.name,home.team.club.name,away.team.name,...,extendedStats.kickins,extendedStats.kickinsPlayon,player.playerId,player.captain,player.playerJumperNumber,player.givenName,player.surname,teamStatus,team.name,season
0,CD_M20190140101,2019-03-21T08:25:00.000+0000,CONCLUDED,Premiership,Round 1,1,MCG,Carlton,Carlton,Richmond,...,NaN,NaN,CD_I996731,False,30,Charlie,Curnow,home,Carlton,2019
1,CD_M20190140101,2019-03-21T08:25:00.000+0000,CONCLUDED,Premiership,Round 1,1,MCG,Carlton,Carlton,Richmond,...,NaN,NaN,CD_I1000953,False,10,Harry,McKay,home,Carlton,2019
2,CD_M20190140101,2019-03-21T08:25:00.000+0000,CONCLUDED,Premiership,Round 1,1,MCG,Carlton,Carlton,Richmond,...,NaN,NaN,CD_I998215,False,43,Will,Setterfield,home,Carlton,2019
3,CD_M20190140101,2019-03-21T08:25:00.000+0000,CONCLUDED,Premiership,Round 1,1,MCG,Carlton,Carlton,Richmond,...,NaN,NaN,CD_I297255,False,11,Mitch,McGovern,home,Carlton,2019
4,CD_M20190140101,2019-03-21T08:25:00.000+0000,CONCLUDED,Premiership,Round 1,1,MCG,Carlton,Carlton,Richmond,...,NaN,NaN,CD_I230202,False,6,Kade,Simpson,home,Carlton,2019


In [69]:
# define aggregation columns and grouping levels
key_stats = ['kicks', 'handballs', 'disposals', 'marks', 'bounces',
       'tackles', 'contestedPossessions', 'uncontestedPossessions',
       'totalPossessions', 'inside50s', 'marksInside50', 'contestedMarks',
       'hitouts', 'onePercenters', 'disposalEfficiency', 'clangers',
       'freesFor', 'freesAgainst', 'rebound50s',
       'goalAssists', 'turnovers', 'intercepts', 'tacklesInside50', 'shotsAtGoal',
       'scoreInvolvements', 'metresGained', 'clearances.centreClearances',
       'clearances.stoppageClearances', 'clearances.totalClearances',
       'extendedStats.effectiveKicks', 'extendedStats.kickToHandballRatio', 'extendedStats.effectiveDisposals',
       'extendedStats.marksOnLead', 'extendedStats.interceptMarks',
       'extendedStats.hitoutsToAdvantage', 'extendedStats.groundBallGets',
       'extendedStats.f50GroundBallGets', 'extendedStats.scoreLaunches',
       'extendedStats.pressureActs', 'extendedStats.defHalfPressureActs',
       'extendedStats.spoils', 'extendedStats.ruckContests',
       'extendedStats.contestDefOneOnOnes', 'extendedStats.contestDefLosses',
       'extendedStats.contestOffOneOnOnes', 'extendedStats.contestOffWins',
       'extendedStats.centreBounceAttendances', 'extendedStats.kickins',
       'extendedStats.kickinsPlayon']
group_by = ['season','round.roundNumber','team.name','player.player.position']
df_cols = key_stats + group_by

In [70]:
# subset to only aggregation + grouping columns
subset_player_stats = player_stats[df_cols]

# apply aggregation and output file
team_stats = subset_player_stats.groupby(group_by).sum()
team_stats.reset_index(inplace=True)
team_stats.to_csv('outputs/team_stats.csv')

In [71]:
# pivot stats out and sum by position for grouped positions
team_stats['idx'] = team_stats.groupby(['season','round.roundNumber','team.name']).cumcount()+1
team_stats_position = team_stats.pivot_table(index=['season','round.roundNumber','team.name','player.player.position'], columns='idx', 
                    values=key_stats, aggfunc='sum')

# set columns to new_names with unstacked columns
df = team_stats_position.sort_index(axis=1, level=1)
df.columns = [f'{x}_{y}' for x,y in df.columns]
df = df.reset_index()

# drop the categrorical positions out
df.drop('player.player.position', axis=1, inplace=True)

# group stats data to team level for all new features
df = df.groupby(['season','round.roundNumber','team.name']).sum()
df.reset_index(inplace=True)

In [72]:
df.to_csv('outputs/team_stats_position.csv')

In [73]:
# get game results to merge into player stats
score_df = game_results[['round.year', 'round.roundNumber', 'match.homeTeam.name', 'match.awayTeam.name', 'homeTeamScore.matchScore.totalScore','awayTeamScore.matchScore.totalScore']]
score_df['round.year'] = score_df['round.year'].astype(str)
score_df['score_diff'] = score_df['homeTeamScore.matchScore.totalScore'] - score_df['awayTeamScore.matchScore.totalScore']

C:\Users\whitehes\AppData\Local\Temp\ipykernel_21384\1582439415.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['round.year'] = score_df['round.year'].astype(str)
C:\Users\whitehes\AppData\Local\Temp\ipykernel_21384\1582439415.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['score_diff'] = score_df['homeTeamScore.matchScore.totalScore'] - score_df['awayTeamScore.matchScore.totalScore']


In [74]:
# merge home team stats for round into score dataframe
home_stats = score_df.merge(df, left_on=['round.year', 'round.roundNumber', 'match.homeTeam.name'],
              right_on=['season', 'round.roundNumber', 'team.name'])
# merge away team stats for round into score+home dataframe
homeaway_stats = home_stats.merge(df, left_on=['round.year', 'round.roundNumber', 'match.awayTeam.name'],
              right_on=['season', 'round.roundNumber', 'team.name'], suffixes=('|HOME','|AWAY'))

# dump to csv
homeaway_stats.to_csv('outputs/model_training_data.csv', index=False)

In [75]:
# collect names of all features with position suffix added to them
n_features = []

for i in range(1,21):
    for feature in key_stats:
        n_features.append(feature+'_'+str(i))

In [76]:
# combine 'HOME' & 'AWAY' suffixed features from combined df to use for subsetting training data
away_n_feat = [feat + '|AWAY' for feat in n_features]
home_n_feat = [feat + '|HOME' for feat in n_features]
tot_n_feat = home_n_feat + away_n_feat

In [77]:
# write list to .py file to be used in model notebook
with open('position_group_features.py','w+') as file:
    file.write('features = ' + str(tot_n_feat))

In [78]:
homeaway_stats[metadata + tot_n_feat + targets].head()

,round.year,round.roundNumber,match.homeTeam.name,match.awayTeam.name,kicks_1|HOME,handballs_1|HOME,disposals_1|HOME,marks_1|HOME,bounces_1|HOME,tackles_1|HOME,...,extendedStats.contestDefOneOnOnes_20|AWAY,extendedStats.contestDefLosses_20|AWAY,extendedStats.contestOffOneOnOnes_20|AWAY,extendedStats.contestOffWins_20|AWAY,extendedStats.centreBounceAttendances_20|AWAY,extendedStats.kickins_20|AWAY,extendedStats.kickinsPlayon_20|AWAY,homeTeamScore.matchScore.totalScore,awayTeamScore.matchScore.totalScore,score_diff
0,2019,1,Carlton,Richmond,8.0,1.0,9.0,3.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64,97,-33
1,2019,1,Collingwood,Geelong Cats,8.0,8.0,16.0,3.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65,72,-7
2,2019,1,Melbourne,Port Adelaide,12.0,2.0,14.0,1.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61,87,-26
3,2019,1,Adelaide Crows,Hawthorn,8.0,12.0,20.0,3.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55,87,-32
4,2019,1,Western Bulldogs,Sydney Swans,11.0,1.0,12.0,4.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,82,65,17
